### Launch the ROS demo

In [1]:
%%bash --bg
trap 'kill $(jobs -p)' EXIT
xterm -e /bin/bash -l -c "roslaunch demo.launch"

Create the controller and run it periodically in a thread

In [2]:
import rospy
import numpy
from threading import Thread
from controller import Controller, iPoseMarker
from interactive_markers.interactive_marker_server import InteractiveMarkerServer
from markers import addMarker, processFeedback

rospy.init_node('ns_demo')
c = Controller()
#c.addConeMarker(iPoseMarker(c.T))
ims = InteractiveMarkerServer('controller')
addMarker(ims, iPoseMarker(c.T), processFeedback(c.setTarget))
ims.applyChanges()

In [3]:
class Worker(Thread):
    def __init__(self, *args, **kwargs): 
        super(Worker, self).__init__(*args, **kwargs) 
        self.stop = False
        self.rate = rospy.Rate(200)
    
    def run(self):
        c.reset()    
        while not rospy.is_shutdown() and not self.stop:
            target = c.targets['pose']
            # position > orientation > joints
            q_delta = c.solve([c.position_task(target, c.T),
                               c.orientation_task(target, c.T),
                               c.joint_task()])
            # pose > joints
            #q_delta = self.solve([c.pose_task(target, c.T), c.joint_task()])
            c.actuate(q_delta)

            self.rate.sleep()
    
try:  # stop previous thread if existing
    t.stop = True
except NameError:
    pass

t = Worker()
t.start()

Create a list of slider widgets, one for each joint, to chose the default pose

In [4]:
import ipywidgets
from ipywidgets import FloatSlider, Layout, Button, Box
joint_widgets = [FloatSlider(min = j.min, max = j.max, step = (j.max-j.min) / 100, description = j.name) \
                 for j in c.robot.active_joints]
joint_weights = [FloatSlider(min = 0, max = 1, step = 0.01, value = 1, description = j.name) \
                 for j in c.robot.active_joints]
task_dimensions = ['x', 'y', 'z', 'rx', 'ry', 'rz']
task_weights = [FloatSlider(min = 0, max = 1, step = 0.01, value = 1, description = name) \
                for name in task_dimensions]

React to slider (value) changes by adapting the default joint pose

In [5]:
def on_joint_preference(event):
    widget = event.owner
    c.preferred_joints[c.joint_msg.name.index(widget.description)] = widget.value

for widget in joint_widgets:
    widget.observe(on_joint_preference, 'value')

In [6]:
def on_joint_weight(event):
    widget = event.owner
    c.joint_weights[c.joint_msg.name.index(widget.description)] = widget.value

for widget in joint_weights:
    widget.observe(on_joint_weight, 'value')

In [7]:
def on_task_weight(event):
    widget = event.owner
    c.cartesian_weights[task_dimensions.index(widget.description)] = widget.value

for widget in task_weights:
    widget.observe(on_task_weight, 'value')

Collect all widgets (sliders and buttons) in a form and display them

In [8]:
layout = Layout(display='flex', flex_flow='column', border='solid 2px', align_items='stretch')
form = Box([Box(w, layout=layout) for w in [joint_widgets, joint_weights, task_weights]])
display(form)

Box(children=(Box(children=(FloatSlider(value=0.0, description='panda_joint1', max=2.9671, min=-2.9671, step=0…